In [5]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fairbet_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [6]:
from order_placed.models import *
from django_pandas.io import read_frame
from django.db.models import F, Q, When, Value, Case, Sum

In [7]:
df = read_frame(Betting.objects.filter(match="Srilanka vs India"))

In [8]:
df

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,7,fairbet,10000.0,Srilanka,BACK,1.85,India,-10000.0,Srilanka vs India,2022-07-04 07:05:52.327891+00:00,2022-07-05 12:46:26.107988+00:00
1,8,fairbet,10000.0,Srilanka,LAY,1.86,India,10000.0,Srilanka vs India,2022-07-04 07:06:03.985142+00:00,2022-07-05 12:46:37.872985+00:00
2,9,fairbet,10000.0,India,BACK,3.10,India,31000.0,Srilanka vs India,2022-07-04 07:07:32.928877+00:00,2022-07-05 12:46:43.930520+00:00
3,10,fairbet,10000.0,India,LAY,3.15,India,-31500.0,Srilanka vs India,2022-07-04 07:08:36.692581+00:00,2022-07-05 12:46:47.919532+00:00


In [9]:
# Client.objects.update(account_type=Case(
#     When(registered_on__lte=a_year_ago,
#          then=Value(Client.PLATINUM)),
#     When(registered_on__lte=a_month_ago,
#          then=Value(Client.GOLD)),
#     default=Value(Client.REGULAR)
# ),)

In [10]:
# betting = Betting.objects.filter(match="Srilanka vs India").values('loss_profit')

In [11]:
# bet = betting.annotate(
#     betting_loss_profit=Case(
#         When(
#             status="BACK",
#             then=F("amount")*0.8,
#         ),
#         default = F("loss_profit"),
#         output_field = models.FloatField()
#     )
# )

In [12]:
# Betting.objects.update(
#     loss_profit=Case(
#         When(status="B",
#              match="Srilanka vs India",
#              winning_team = "Not declared yet",
#              then=F("amount")*F("odds") - F("amount")),
#         When(status="L",
#              match="Srilanka vs India",
#              winning_team = "Not declared yet",
#              then=F("amount")* -F("odds") + F("amount")),
#         default=F("loss_profit")
# )
#      )

In [16]:
from order_placed.models import *
from django_pandas.io import read_frame
from django.db.models import F, Q, When, Value, Case, Sum

In [17]:
winningTeam = "Srilanka"
matchName = "Srilanka vs India"

In [18]:
betting_instance = Betting.objects.filter(
        match = matchName
    ).update(
        loss_profit = Case(
            When(
                status = "B",
                bet_on_team = winningTeam,
                winning_team = winningTeam,
                then = F('amount') * F("odds")),
            When(
                ~Q(bet_on_team = winningTeam),
                status = "B",
                winning_team = winningTeam,
                then = -F('amount')),
             When(
                status = "L",
                bet_on_team = winningTeam,
                winning_team = winningTeam,
                then = F('amount') * - F("odds")),
             When(
                 ~Q(bet_on_team = winningTeam),
                status = "L",
                winning_team = winningTeam,
                then = F('amount')),
            default=F("loss_profit")
            )
        )

In [131]:
bettings = Betting.objects.filter(loss_profit__gt=0)

In [132]:
bettings

<BulkUpdateOrCreateQuerySet [<Betting: Betting object (7)>, <Betting: Betting object (10)>]>